In [18]:
from modules import find_csv_urls
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
from yaml import dump

In [28]:
remote_data_url = r"https://github.com/ONSdigital/sdg-data/tree/develop/data"


TODO: include these key:value pairs for every dataset
fix_headers : False
standardise_cells : True 
fill_gaps : True

In [31]:
# Yaml dump version
csv_gen = find_csv_urls(remote_data_url)
yam_dict={'generic_overrides':
          {'FILL_NA' : 'generic_gap_filler_value',
          'OldValue1' : 'NewValue1',
          'OldValue2' : 'NewValue2',
          'OldValue3' : 'NewValue3'}}
for url in csv_gen:
    if "no data for this indicator yet" in str(bs(requests.get(url).text)):
        continue
    df = pd.read_csv(url)
#     import ipdb; ipdb.set_trace()
    dataset_specific_values_dict= {}
    dataset_specific_values_dict['fix_headers'] = False
    dataset_specific_values_dict['standardise_cells'] = True
    dataset_specific_values_dict['fill_gaps'] = True
    for column in df.columns:
        if column == 'Value':
            continue
        dataset_specific_values_dict[column]= {'to' : f'correct_header_for_{column.replace(" ","_")}',
            'FILL_NA' : f'{column.replace(" ","_")}_gap_filler_value',
            'OldValue1' : 'NewValue1',
            'OldValue2' : 'NewValue2',
            'OldValue3' : 'NewValue3'}
    yam_dict[url] = dataset_specific_values_dict

with open('overrides_dict.yaml', 'w') as yam:
    dump(yam_dict, yam)